# Doctor Semmelweis discovery

## Motivation

The data that Doctor Semmelweis collected is very well documented, so it is a valuable resource for learning how to perform statistical tests comparing two groups. 

The knowledge gained in this way can then be used to perform similar tests on other data sets. For example, it can help determine whether the risk of heart attack depends on the degree of obesity or whether smoking cigarettes increases the chance of lung cancer. In technical issues, for example, it can determine whether the same product produced on machine A is different from that produced on machine B.

### Main goal

To determine by statistical methods whether:

+ there is a difference between the mortality rate in the first clinic, where doctors and students delivered babies, and the second clinic, where only midwives did so,
+ the mortality rate decreased significantly in Doctor Semmelweis' clinic after he recommended that the attendants wash their hands. <br /> 
Mandatory hand washing before delivered babies: 1 June 1847. 

### Side goal

+ Better understanding of Python and statistical methods.

# Libraries and data loading

In [1]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import numpy as np
import scipy.stats as st

In [2]:
plt.rcParams['figure.figsize'] = [10, 8]  # for size of figures

In [3]:
df_1 = pd.read_csv("monthly_deaths.csv")
df_2 = pd.read_csv("yearly_deaths_by_clinic.csv")
print("df_1:", df_1.shape, "df_2: ", df_2.shape)

df_1: (98, 3) df_2:  (12, 4)


# Data preview and manipulations

In [4]:
print("df_1 dtypes:")
print(df_1.dtypes)
print()
print("df_2 dtypes:")
print(df_2.dtypes)


df_1 dtypes:
date      object
births     int64
deaths     int64
dtype: object

df_2 dtypes:
year       int64
births     int64
deaths     int64
clinic    object
dtype: object


Since the data type of the *date* variable (df_1) and the *year* variable (df_2) are in the wrong format, we will perform a conversion.

In [5]:
df_1["date"] = pd.to_datetime(df_1.date)
df_2["year"] = pd.to_datetime(df_2.year).dt.year

Since the number of births in a given period is not constant, we need to create an additional column with the proportion of deaths in order to compare mortality due to puerperal fever.

$ births \rightarrow 1 $ <br />
$ deaths \rightarrow x $

If we divide deaths by births we get a number between 0 and 1 indicating what proportion of births are those that ended with a death.

In [6]:
df_1["proportion_deaths"] = df_1["deaths"]/df_1["births"]
df_2["proportion_deaths"] = df_2["deaths"]/df_2["births"]

We will add an additional variable to the database to specify whether hand washing was mandatory on a given date.

In [17]:
handwashing = []
mhw_day = pd.to_datetime("1847/6/1")

for row in df_1["date"]:
    if row >= mhw_day: handwashing.append("mandatory")
    else: handwashing.append("not mandatory")

df_1["handwashing"] = handwashing

# Confidence Intervals for means

## According to the clinic

Since we are interested in the mean of the proportion of deaths, we can calculate confidence intervals for the mean.

$ \bar{x} \pm t\frac{s}{\sqrt{n}} $ 

where: $\bar{x}$ - sample mean, $t$ - the value of the chosen distribution with $n-1$ degrees of freedom (for small samples t-distribution, for large samples normal distribution), $s$ - sample standard deviation, $n$ - sample size

In Python $\frac{s}{\sqrt{n}}$ can be calculated manually as `(np.std(x, ddof=1)/np.sqrt(np.size(x)))` or usung the function `sem(x)` from *scipy.stats*

In [7]:
degreesFreedom_c1 = len(df_2[df_2["clinic"] == "clinic 1"])-1
degreesFreedom_c2 = len(df_2[df_2["clinic"] == "clinic 2"])-1
alpha = 0.95

clinic_1 = df_2.loc[df_2["clinic"] == "clinic 1", ["proportion_deaths"]]
clinic_2 = df_2.loc[df_2["clinic"] == "clinic 2", ["proportion_deaths"]]

ci_1 = st.t.interval(alpha=alpha, 
                     df=degreesFreedom_c1,
                     loc = np.mean(clinic_1),
                     scale = st.sem(clinic_1))

ci_2 = st.t.interval(alpha=alpha,
                     df=degreesFreedom_c2,
                     loc=np.mean(clinic_2),
                     scale=st.sem(clinic_2))

print("Confidence intervals for mean of proportion deaths in clinic 1:")
print("mean: ", np.mean(clinic_1))
print("Interval:", ci_1[0], " - ", ci_1[1])
print()
print("Confidence intervals for mean of proportion deaths in clinic 2:")
print("mean: ", np.mean(clinic_2))
print("Interval:", ci_2[0], " - ", ci_2[1])

Confidence intervals for mean of proportion deaths in clinic 1:
mean:  proportion_deaths    0.098505
dtype: float64
Interval: [0.06409911]  -  [0.13291143]

Confidence intervals for mean of proportion deaths in clinic 2:
mean:  proportion_deaths    0.0404
dtype: float64
Interval: [0.0167942]  -  [0.06400567]


> There is a 95% chance that the confidence interval of [0.064, 0.133] contains the true population mean of proportion deaths for clinic 1.

> There is a 95% chance that the confidence interval of [0.016, 0.064] contains the true population mean of proportion deaths for clinic 2.

Comparing the two sample means, it can be concluded that the proportion of deaths in the clinic where births were delivered by midwives was lower than in the clinic where births were also delivered by doctors and students.

In order to formulate a final judgment, an appropriate statistical test (*included in the next section*) must be performed.

## According to the mandatory hand washing

In [13]:
df_1.handwashing.value_counts()

not mandatory    76
mandatory        22
Name: handwashing, dtype: int64

Since we have less than 30 observations for mandatory handwashing we will use the t-distribution further on.

In [14]:
df_1.groupby("handwashing").proportion_deaths.mean()

handwashing
mandatory        0.021093
not mandatory    0.105050
Name: proportion_deaths, dtype: float64

In [16]:
degreesFreedom_mhwN = len(df_1[df_1["handwashing"] == "not mandatory"])-1
degreesFreedom_mhwY = len(df_1[df_1["handwashing"] == "mandatory"])-1
alpha = 0.95

handwashing_N = df_1.loc[df_1["handwashing"] == "not mandatory", ["proportion_deaths"]]
handwashing_Y = df_1.loc[df_1["handwashing"] == "mandatory", ["proportion_deaths"]]

ci_N = st.t.interval(alpha=alpha,
                     df=degreesFreedom_mhwN,
                     loc=np.mean(handwashing_N),
                     scale=st.sem(handwashing_N))

ci_Y = st.t.interval(alpha=alpha,
                     df=degreesFreedom_mhwY,
                     loc=np.mean(handwashing_Y),
                     scale=st.sem(handwashing_Y))

print("Confidence intervals for mean of proportion deaths for not mandatory handwashing:")
print("mean: ", np.mean(handwashing_N))
print("Interval:", ci_N[0], " - ", ci_N[1])
print()
print("Confidence intervals for mean of proportion deaths for mandatory handwashing:")
print("mean: ", np.mean(handwashing_Y))
print("Interval:", ci_Y[0], " - ", ci_Y[1])


Confidence intervals for mean of proportion deaths for not mandatory handwashing:
mean:  proportion_deaths    0.10505
dtype: float64
Interval: [0.08888773]  -  [0.12121224]

Confidence intervals for mean of proportion deaths for mandatory handwashing:
mean:  proportion_deaths    0.021093
dtype: float64
Interval: [0.01435474]  -  [0.02783201]


> There is a 95% chance that the confidence interval of [0.087, 0.122] contains the true population mean of proportion deaths for not mandatory handwashing time period.

> There is a 95% chance that the confidence interval of [0.014, 0.028] contains the true population mean of proportion deaths for mandatory handwashing time period.

By comparing the two sample means, it can be believed that the enforcement of mandatory handwashing significantly reduced the average death proportion.

In order to formulate a final judgment, an appropriate statistical test (*included in the next section*) must be performed.